In [1]:
import pyscf
import pyscf.tools
import scipy
import numpy as np

from orbitalpartitioning import *

/Users/arnab/arnab/workspace/.venv/lib/python3.9/site-packages/pyscf/dft/libxc.py:771: UserWarning: Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, corresponding to the original definition by Stephens et al. (issue 1480) and the same as the B3LYP functional in Gaussian. To restore the VWN5 definition, you can put the setting "B3LYP_WITH_VWN5 = True" in pyscf_conf.py
  warnings.warn('Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, '


In [2]:
molecule = """
Cu -3.019630 -0.007541 -0.018036
Mn 2.414767 -0.023478 0.030690
N -1.567623 1.238448 -0.410772
N -1.580212 -1.277148 0.270959
N -4.477455 -1.377388 0.234355
N -4.445900 1.415487 -0.119243
C -0.396910 0.713913 -0.231658
C -0.402291 -0.751244 0.193785
O 0.719754 1.274368 -0.382229
O 0.709685 -1.305693 0.409195
N 4.035049 -1.399594 0.787214
N 2.805466 -1.226444 -1.858758
N 2.810332 1.223361 1.887095
N 3.835899 1.504798 -0.838902
H -5.344417 -1.227673 -0.285776
H -4.761790 -1.469833 1.213052
H -4.766482 1.576647 -1.077764
H -5.296267 1.268483 0.427975
H -4.167500 -2.313239 -0.035954
H -1.585917 -2.255151 0.559517
H -1.551477 2.213879 -0.705967
H -4.086023 2.320991 0.189752
H 3.778153 1.430066 2.139229
H 2.369771 2.137275 1.758658
H 2.395577 0.869641 2.752500
H 4.689943 1.721736 -0.321823
H 3.329935 2.393266 -0.885199
H 4.156909 1.359875 -1.798233
H 3.771503 -1.400275 -2.140808
H 2.359005 -0.860138 -2.702860
H 2.392501 -2.155451 -1.746877
H 3.745933 -2.377479 0.704362
H 4.295729 -1.309775 1.771472
H 4.925795 -1.353260 0.288437
"""

basis = 'def2-svp'
pymol = pyscf.gto.Mole(
        atom    =   molecule,
        symmetry=   True,
        spin    =   6, # number of unpaired electrons
        charge  =   2,
        basis   =   basis)


pymol.build()
print("symmetry: ",pymol.topgroup)
# mf = pyscf.scf.UHF(pymol).x2c()
mf = pyscf.scf.ROHF(pymol).newton()
mf.verbose = 4
mf.conv_tol = 1e-8
mf.conv_tol_grad = 1e-5
mf.chkfile = "scf.fchk"
mf.init_guess = 'chkfile'         # Load initial guess from chkfile
mf.kernel()  
mf.run(max_cycle=200)

print(" Hartree-Fock Energy: %12.8f" % mf.e_tot)
# mf.analyze()

# Get data
F = mf.get_fock()
C = mf.mo_coeff
S = mf.get_ovlp()
ndocc = mf.nelec[1]
nsing = mf.nelec[0] - ndocc
nvirt = mf.mol.nao - ndocc - nsing

# Just use alpha orbitals
Cdocc = mf.mo_coeff[:,0:ndocc]
Csing = mf.mo_coeff[:,ndocc:ndocc+nsing]
Cvirt = mf.mo_coeff[:,ndocc+nsing:ndocc+nsing+nvirt]

nbas = Cdocc.shape[0]

symmetry:  C1




******** <class 'pyscf.soscf.newton_ah.SecondOrderROHF'> ********
method = SecondOrderROHF
initial guess = chkfile
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
SCF conv_tol = 1e-08
SCF conv_tol_grad = 1e-05
SCF max_cycles = 50
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = scf.fchk
max_memory 4000 MB (current use 0 MB)
num. doubly occ = 75  num. singly occ = 6
******** <class 'pyscf.scf.rohf.ROHF'> Newton solver flags ********
SCF tol = 1e-08
conv_tol_grad = 1e-05
max. SCF cycles = 50
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = scf.fchk
max_cycle_inner = 12
max_stepsize = 0.05
ah_start_tol = 1e+09
ah_level_shift = 0
ah_conv_tol = 1e-12
ah_lindep = 1e-14
ah_start_cycle = 1
ah_max_cycle = 40
ah_grad_trust_region = 2.5
kf_interval = 4
kf_trust_region = 5
canonicalization = True
max_memory 4000 MB (current use 0 MB)
  HOMO = -0.203588489487727  LUMO = -0

In [3]:
mf.analyze()

**** SCF Summaries ****
Total Energy =                       -3459.962404847846301
Nuclear Repulsion Energy =            1775.876597056283572
One-electron Energy =                -8287.868501592092798
Two-electron Energy =                 3052.029499687962925
**** MO energy ****
                Roothaan           | alpha              | beta
MO #1   energy= -329.29286466763   | -329.293191446981  | -329.292537888278  occ= 2
MO #2   energy= -240.749933836294  | -240.750076864311  | -240.749790808278  occ= 2
MO #3   energy= -41.3390195151171  | -41.3587427344828  | -41.3192962957513  occ= 2
MO #4   energy= -36.1541501776574  | -36.1806244296538  | -36.127675925661   occ= 2
MO #5   energy= -36.154029602239   | -36.1803814201823  | -36.1276777842957  occ= 2
MO #6   energy= -36.1371633972091  | -36.1415530546658  | -36.1327737397527  occ= 2
MO #7   energy= -29.3262621219746  | -29.3393738137364  | -29.3131504302127  occ= 2
MO #8   energy= -25.0417659838182  | -25.0642647098065  | -25.0192672

((array([1.99999899e+00, 1.99999308e+00, 1.99996281e+00, 8.30380012e-01,
         3.10492051e-01, 1.99999267e+00, 1.99999371e+00, 1.99999772e+00,
         1.99983619e+00, 1.99985658e+00, 1.99996396e+00, 1.77671388e-02,
         1.61649812e-02, 5.69691406e-03, 1.15325545e+00, 1.99045446e+00,
         1.98508905e+00, 1.95935938e+00, 1.99305099e+00, 9.38699546e-03,
         3.87637865e-03, 4.52938644e-03, 3.82647049e-03, 4.57587944e-03,
         5.38883504e-05, 7.58276896e-06, 1.07213145e-05, 1.67210325e-05,
         1.02042723e-05, 5.21941045e-06, 4.94022446e-05, 1.99999828e+00,
         1.99999125e+00, 1.99994718e+00, 4.26114596e-01, 1.56954419e-02,
         1.99998969e+00, 1.99999117e+00, 1.99999319e+00, 1.99963763e+00,
         1.99962932e+00, 1.99971100e+00, 1.37133724e-02, 1.11713303e-02,
         1.78295582e-02, 2.97291905e-01, 5.41930476e-01, 8.25128744e-01,
         1.80426036e+00, 1.74796333e+00, 2.18492278e-03, 2.32116543e-03,
         1.72909096e-03, 7.53396565e-04, 9.23728610

# Define Fragments by AOs

In [26]:
# Find AO's corresponding to atoms
full = []
frag1 = []
frag2 = []
frag3 = []
frag4 = []
frag5 = []
frag6 = []
"""
pop of  0 Cu 4s           0.83038
pop of  0 Cu 5s           0.31049
pop of  0 Cu 3dxy         1.15326
pop of  0 Cu 3dyz         1.99045
pop of  0 Cu 3dz^2        1.98509
pop of  0 Cu 3dxz         1.95936
pop of  0 Cu 3dx2-y2      1.99305
pop of  0 Cu 4dxy         0.00939
pop of  0 Cu 4dyz         0.00388
pop of  0 Cu 4dz^2        0.00453
pop of  0 Cu 4dxz         0.00383
pop of  0 Cu 4dx2-y2      0.00458

pop of  1 Mn 4s           0.42611
pop of  1 Mn 5s           0.01570
pop of  1 Mn 3dxy         0.29729
pop of  1 Mn 3dyz         0.54193
pop of  1 Mn 3dz^2        0.82513
pop of  1 Mn 3dxz         1.80426
pop of  1 Mn 3dx2-y2      1.74796
pop of  1 Mn 4dxy         0.00218
pop of  1 Mn 4dyz         0.00232
pop of  1 Mn 4dz^2        0.00173
pop of  1 Mn 4dxz         0.00075
pop of  1 Mn 4dx2-y2      0.00092

pop of  2 N 2px           1.49063
pop of  2 N 2py           1.32542
pop of  2 N 2pz           1.03344

pop of  6 C 2px           0.83933
pop of  6 C 2py           0.95746
pop of  6 C 2pz           0.98082

pop of  8 O 2px           1.53962
pop of  8 O 2py           1.79193
pop of  8 O 2pz           1.51099


"""

for ao_idx,ao in enumerate(mf.mol.ao_labels(fmt=False)):
    if ao[0] == 0:
        if ao[2] in ("3d","4d","4s","5s"):
            frag1.append(ao_idx)
            full.append(ao_idx)
    elif ao[0] == 1:
        if ao[2] in ("3d","4d","4s"):
            frag2.append(ao_idx)
            full.append(ao_idx)
    elif ao[0] == 2:
        if ao[2] in ("2p"):
            # if ao[3] in ("z"):
            frag3.append(ao_idx)
            full.append(ao_idx)
    elif ao[0] == 6:
        if ao[2] in ("2p"):
            # if ao[3] in ("z"):
            frag3.append(ao_idx)
            full.append(ao_idx)
    elif ao[0] == 8:
        if ao[2] in ("2p"):
            # if ao[3] in ("z"):
            frag3.append(ao_idx)
            full.append(ao_idx)
    elif ao[0] == 3:
        if ao[2] in ("2p"):
            # if ao[3] in ("z"):
                frag4.append(ao_idx)
                full.append(ao_idx)
    elif ao[0] == 7:
        if ao[2] in ("2p"):
            # if ao[3] in ("z"):
                frag4.append(ao_idx)
                full.append(ao_idx)
    elif ao[0] == 9:
        if ao[2] in ("2p"):
            # if ao[3] in ("z"):
                frag4.append(ao_idx)
                full.append(ao_idx)
    
# Find AO's corresponding to atoms

frags_all=[frag1,frag2,frag3,frag4]
print(frags_all)
print(full)


[[3, 4, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23], [34, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54], [65, 66, 67, 121, 122, 123, 149, 150, 151], [79, 80, 81, 135, 136, 137, 163, 164, 165]]
[3, 4, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 34, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 65, 66, 67, 79, 80, 81, 121, 122, 123, 135, 136, 137, 149, 150, 151, 163, 164, 165]


In [27]:
# Define projectors
X = scipy.linalg.sqrtm(S)
X = np.eye(nbas) 
Pfull = X[:,full]  # non-orthogonal
Pf = []
for f in frags_all:
    Pf.append(X[:,f])


# Project MOs onto all fragments
For each orbital block (Docc, Sing, Virt), project each subspace onto the full list of fragment AOs. This will determine our full active space.

In [28]:
(Oact, Sact, Vact), (Cenv, Cerr, _) = svd_subspace_partitioning((Cdocc, Csing, Cvirt), Pfull, S)
print(Cerr.shape)
assert(Cerr.shape[1] == 0)
Cact = np.hstack((Oact, Sact, Vact))

 Partition  330 orbitals into a total of   41 orbitals
            Index   Sing. Val. Space       
                0   0.99999682            2*
                1   0.99995171            0*
                2   0.99987401            0*
                3   0.99984324            0*
                4   0.99982063            0*
                5   0.99962792            0*
                6   0.99960362            2*
                7   0.99927945            2*
                8   0.99866003            0*
                9   0.99677442            1*
               10   0.99533733            2*
               11   0.99520806            2*
               12   0.99441566            2*
               13   0.99262280            2*
               14   0.99076175            2*
               15   0.98940427            2*
               16   0.98310026            0*
               17   0.98255684            2*
               18   0.98220127            0*
               19   0.97344141            0*
 

In [29]:
# Project active orbitals onto fragments
init_fspace = []
clusters = []
Cfrags = []
orb_index = 1


for fi,f in enumerate(frags_all):
    print()
    print(" Fragment: ", f)
    (Of, Sf, Vf), (_, _, _) = svd_subspace_partitioning((Oact, Sact, Vact), Pf[fi], S)
    Cfrags.append(np.hstack((Of, Sf, Vf)))
    ndocc_f = Of.shape[1]
    init_fspace.append((ndocc_f+Sf.shape[1], ndocc_f))
    nmof = Of.shape[1] + Sf.shape[1] + Vf.shape[1]
    clusters.append(list(range(orb_index, orb_index+nmof)))
    orb_index += nmof



# Orthogonalize Fragment orbitals
Cfrags = sym_ortho(Cfrags, S)
Cfrags = canonicalize(Cfrags, F)
Cact = np.hstack(Cfrags)

# Write Molden files for visualization
pyscf.tools.molden.from_mo(mf.mol, "Pfull.molden", Pfull)
pyscf.tools.molden.from_mo(mf.mol, "Cact.molden", Cact)
pyscf.tools.molden.from_mo(mf.mol, "Cenv.molden", Cenv)
for i in range(len(frags_all)):
    pyscf.tools.molden.from_mo(mf.mol, "Cfrag%i.molden"%i, Cfrags[i])

print(" init_fspace = ", init_fspace)
print(" clusters    = ", clusters)





 Fragment:  [3, 4, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]
 Partition   41 orbitals into a total of   12 orbitals
            Index   Sing. Val. Space       
                0   0.99993054            0*
                1   0.99979253            0*
                2   0.99935159            0*
                3   0.99903269            0*
                4   0.98463179            2*
                5   0.98118181            2*
                6   0.97960059            2*
                7   0.97736108            2*
                8   0.94414838            1*
                9   0.83865329            1*
               10   0.82317663            0*
               11   0.77997116            2*
               12   0.49164554            0
               13   0.44284274            2
               14   0.19947016            0
               15   0.15551347            1
               16   0.13461996            1
               17   0.07352236            0
               18   0.04998075       

# Make Integrals

In [30]:
print(Cenv.shape)
print(Cact.shape)
d1_embed = 2 * Cenv @ Cenv.T

h0 = pyscf.gto.mole.energy_nuc(mf.mol)
h  = pyscf.scf.hf.get_hcore(mf.mol)
j, k = pyscf.scf.hf.get_jk(mf.mol, d1_embed, hermi=1)

print(h.shape)
h0 += np.trace(d1_embed @ ( h + .5*j - .25*k))

h = Cact.T @ h @ Cact;
j = Cact.T @ j @ Cact;
k = Cact.T @ k @ Cact;
nact = h.shape[0]

h2 = pyscf.ao2mo.kernel(pymol, Cact, aosym="s4", compact=False)
h2.shape = (nact, nact, nact, nact)
# The use of d1_embed only really makes sense if it has zero electrons in the
# active space. Let's warn the user if that's not true

S = pymol.intor("int1e_ovlp_sph")
n_act = np.trace(S @ d1_embed @ S @ Cact @ Cact.T)
if abs(n_act) > 1e-8 == False:
    print(n_act)
    error(" I found embedded electrons in the active space?!")

h1 = h + j - .5*k;

(330, 57)
(330, 41)
(330, 330)


In [31]:
np.save("ints_h0_41", h0)
np.save("ints_h1_41", h1)
np.save("ints_h2_41", h2)
np.save("mo_coeffs_41", Cact)
np.save("overlap_mat_41", S)

Pa = mf.make_rdm1()[0]
Pb = mf.make_rdm1()[1]
np.save("Pa_41", Cact.T @ S @ Pa @ S @ Cact)
np.save("Pb_41", Cact.T @ S @ Pb @ S @ Cact)

In [32]:
import numpy as np
Ccmf = np.load("Ccmf_32.npy")
pyscf.tools.molden.from_mo(mf.mol, "Ccmf_32.molden", Ccmf)
